# DataFrame Pre Processing

In [2]:
import pandas as pd
import numpy as np

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url)[0]
df.drop(df.loc[df['Borough'] == 'Not assigned'].index, inplace = True)
df.reset_index(drop = True, inplace = True)

In [17]:
d_flag = False
for i,a in enumerate(df['Postal code']):
    for j,b in enumerate(df['Postal code']):
        if j > i:
            if a==b:
                print(a,b,'are duplicates')
                d_flag = True
if d_flag == False:
    print('No duplicates in the dataframe')

No duplicates in the dataframe


In [5]:
df.loc[df['Postal code'] == 'M5A']

,Postal code,Borough,Neighborhood
2,M5A,Downtown Toronto,Regent Park / Harbourfront


Neighborhoods are already combined but splitted by "/" instead of ","

In [16]:
df['Neighborhood'] = df['Neighborhood'].str.replace('/',',')

In [33]:
print(df[['Neighborhood']].isna().values.any())
print(df.isna().values.any())
print((df['Neighborhood'] == 'Not assigned').values.any())

False
False
False


No NAN or Not assigned values

In [70]:
print(df.shape)

(103, 5)


# Adding Latitude and Longitude

In [41]:
latlon = pd.read_csv('Geospatial_Coordinates.csv')

In [67]:
# create lat and lon columns in df 
df['Latitude'] = np.nan
df['Longitude'] = np.nan
df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,NaN,NaN
1,M4A,North York,Victoria Village,NaN,NaN
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",NaN,NaN
3,M6A,North York,"Lawrence Manor , Lawrence Heights",NaN,NaN
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",NaN,NaN


Assign values from latlot dataframe to df 

In [68]:
for i, row in df.iterrows():
    df.at[i,'Latitude']=latlon.loc[latlon['Postal Code'] == row['Postal code']].Latitude
    df.at[i,'Longitude'] = latlon.loc[latlon['Postal Code'] == row['Postal code']].Longitude

In [69]:
df.head(10)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
